# Objective

- csv 파일을 읽어서 torchtext를 사용하여 데이터를 신경망에 입력가능한 꼴로 바꾸기
(Field, Iterator, train,test, evaluation and prediction)
- base line으로 Naive Bayes classification 구현
- 한국어 데이터 전처리를 위한 함수를 만들고 이를 torchtext에 통합하기 
- 제시된 여러 모델을 사용하여(transformers 제외) 성능을 향상 시키기
- training, evaluation 한 것을 test 데이터에 적용하여 성능을 보이기.
- predict를 사용하여 제시된 기사들의 분류 결과를 보이기

- 참고 사이트
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text

## 내용

- 첨부된 BalancedNewsCorpus_train.csv, BalancedNewsCorpus_test.csv는 국어원 뉴스 자료에서 9개 분야의 신문별 균형을 맞춘 자료로, 학습용 9,000개 시험용 1800 자료가 있는 파일이다.
- 이 파일을 가지고 https://github.com/bentrevett/pytorch-sentiment-analysis 에 있는 pytorch sentiment analysis의 방법을 따라 한국어 뉴스기사 분류기를 만들어라
- 한국어 선처리를 위해 함수를 만들어 이를 torchText에 통합하여 사용. preprocessing은 다양한 방법으로 가능함.
- baseline으로 Naive Bayes를 사용하고 Neural Network를 사용하는 모델이 얼마나 더 성능의 향상을 이루는지 보여라..
- https://github.com/bentrevett/pytorch-sentiment-analysis 에 제시된 방법 중 가장 성능이 좋은 방법을 사용할 수 있음. **단 이 과제에서는 외부 임베딩과, transformers를 사용하는 방법은 적용하지 말것**
- Evaluation, Test 성능을 정리하고, 이렇게 학습한 모델로 제시된 User Input에서 제시된 문장의 출력과 정답을 비교 분석하라.


## 1. Data Preprocessing

In [1]:
# 필요 Package Import
import os
from io import StringIO
import hanja
import re
import random
import time
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from konlpy.tag import Mecab
from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

random.seed(319)
np.random.seed(319)
torch.manual_seed(319)
torch.backends.cudnn.deterministic = True

In [2]:
# Preprocessing
# 한자는 hanja를 통해 번역, 그외 영어나 기호는 삭제

train_data_path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_train.csv'
test_data_path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_test.csv'
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

def preprocess(text):
    text = hanja.translate(text, 'substitution')
    korean_list = re.compile('[ㄱ-ㅎ 가-힣]+').findall(text)
    text = ' ' + ' '.join([word for word in korean_list]) + " "
    return text


# Doing the preprocessing
train_data = train_df['News'].apply(lambda x: preprocess(x))
test_data = test_df['News'].apply(lambda x: preprocess(x))

# Extracting Label
train_labels = train_df['Topic']
test_labels = test_df['Topic']


# Download preprocessed data csv
temp_train_df = pd.concat([train_data, train_labels], axis=1)
temp_train_df, temp_valid_df = train_test_split(temp_train_df, test_size=0.2)

temp_test_df = pd.concat([test_data, test_labels], axis=1)
path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_'

temp_train_data_path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_train_temp.csv'
temp_valid_data_path = path + 'valid_temp.csv'
temp_test_data_path = path + 'test_temp.csv'

temp_train_df.to_csv(temp_train_data_path, index=False)
temp_valid_df.to_csv(temp_valid_data_path, index=False)
temp_test_df.to_csv(temp_test_data_path, index=False)
print('train :', {temp_train_df.shape[0]}, 'valid :', {temp_valid_df.shape[0]}, 'test :', {temp_test_df.shape[0]})


train : {7200} valid : {1800} test : {1800}


## 2. Naive Bayes 

In [3]:
# Dataloading
path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_'
temp_train_path = path + 'train_temp.csv'
temp_valid_path = path + 'valid_temp.csv'
temp_test_path = path + 'test_temp.csv'

temp_train_data = pd.read_csv(temp_train_path)
temp_valid_data = pd.read_csv(temp_valid_path)
temp_test_data = pd.read_csv(temp_test_path)

In [4]:
# space 단위 tokenizing 함수
def preprocess_text_split(text):
    return text.split()

# space 단위 tokenizing을 이용한 전처리
vectorizer = CountVectorizer(analyzer=preprocess_text_split)

# Train X & Y 생성
train_data_nb = vectorizer.fit_transform(temp_train_data['News'])
train_label_nb = temp_train_data['Topic']

# Valid X & Y 및 Test X & Y 생성
valid_data_nb = vectorizer.transform(temp_valid_data['News'])
valid_label_nb = temp_valid_data['Topic']
test_data_nb = vectorizer.transform(temp_test_data['News'])
test_label_nb = temp_test_data['Topic']

In [5]:
# Fitting
NB_split = MultinomialNB()
NB_split.fit(train_data_nb, train_label_nb)

# Results 
predictions_train = NB_split.predict(train_data_nb)
print('  - Training accuracy = {}'.format(
        accuracy_score(predictions_train, train_label_nb) * 100)
     )
predictions_valid = NB_split.predict(valid_data_nb)
print('  - Validation accuracy = {}'.format(
        accuracy_score(predictions_valid, valid_label_nb) * 100)
     )
predictions_test = NB_split.predict(test_data_nb)
print('  - Test accuracy = {}'.format(
        accuracy_score(predictions_test, test_label_nb) * 100)
     )

  - Training accuracy = 97.68055555555556
  - Validation accuracy = 74.27777777777777
  - Test accuracy = 74.27777777777777


### Naive bayes 결과 요약
- 단순히 regex만 처리하여 한글만 남겼을 때의 test set에서의 정확도는 약 74.8%
- 따라서 74.8%를 baseline으로 사용할 계획임.

## CNN모델 구현하기

In [6]:
# 3. 전처리 데이터를 모델에 사용가능하도록 수정한다

# 전처리 끝난 file path
path = './BalancedNewsCorpusShuffled/BalancedNewsCorpus_'
temp_train_data_path = path + 'train_temp.csv'
temp_valid_data_path = path + 'valid_temp.csv'
temp_test_data_path = path + 'test_temp.csv'

tokenizer = Mecab()

TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  batch_first=True,
                  )

LABEL = data.LabelField()

In [7]:
train_data, test_data = TabularDataset.splits(
        path='./', train=temp_train_data_path, test=temp_test_data_path, format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

valid_data, _ = TabularDataset.splits(
        path='./', train=temp_valid_data_path, test=temp_test_data_path, format='csv',
        fields=[('text', TEXT), ('label', LABEL)], skip_header=True)


In [8]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print('TEXT 집합의 크기: {0}'.format(len(TEXT.vocab)))

TEXT 집합의 크기: 66337


CNN 모델 부분

In [9]:
# CNN Model structure
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout):        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [sent len, batch size]              
        embedded = self.embedding(text)                
        #embedded = [batch size, sent len, emb dim]        
        embedded = embedded.unsqueeze(1)        
        #embedded = [batch size, 1, sent len, emb dim]        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]        
        #pooled_n = [batch size, n_filters]        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]            
        return self.fc(cat)

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [11]:
def train(model, iterator, optimizer, criterion):   
    
    epoch_loss = 0
    epoch_acc = 0 
    
    model.train() 
    
    for batch in iterator:      
        
        optimizer.zero_grad()        
        predictions = model(batch.text)  
        
        loss = criterion(predictions, batch.label)    
        
        acc = categorical_accuracy(predictions, batch.label)   
        
        loss.backward()        
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()      
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
def evaluate(model, iterator, criterion):   
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()    
    
    with torch.no_grad():    
        
        for batch in iterator:
            predictions = model(batch.text)            
            loss = criterion(predictions, batch.label)            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [14]:
input_dim=len(TEXT.vocab)
embedding_dim=300
filter_sizes=[2,3,4]
output_dim=len(LABEL.vocab)
n_filters=120
dropout=0.5

model=CNN(input_dim,embedding_dim, n_filters, filter_sizes, output_dim, dropout)

batch_size=64
optimzer=optim.Adam(model.parameters())
criterion=nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
criterion=criterion.to(device)


In [15]:


train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = batch_size,
    sort=False,
    device = device)

In [16]:
n_epoch=30

best_val_loss=float('inf')

for i in range(n_epoch):
    train_loss, train_acc= train(model,train_iterator,optimzer,criterion)
    val_loss, val_acc = evaluate(model, valid_iterator, criterion)
    
    if val_loss < best_val_loss:
        best_val_loss= val_loss
        torch.save(model.state_dict(),'best_cnn_clf.pt')
        
    print('epoch #: {0}'.format(i+1))
    print('Train loss is: {:.2f}'.format(train_loss))
    print('Train accuracy is: {:.2f}'.format(train_acc))
    print('val loss is: {:.2f}'.format(val_loss))
    print('val accuracy is: {:.2f}'.format(val_acc))
    print('-'*30)

epoch #: 1
Train loss is: 2.09
Train accuracy is: 0.29
val loss is: 1.19
val accuracy is: 0.60
------------------------------
epoch #: 2
Train loss is: 1.27
Train accuracy is: 0.57
val loss is: 0.99
val accuracy is: 0.67
------------------------------
epoch #: 3
Train loss is: 1.04
Train accuracy is: 0.65
val loss is: 0.86
val accuracy is: 0.71
------------------------------
epoch #: 4
Train loss is: 0.91
Train accuracy is: 0.69
val loss is: 0.79
val accuracy is: 0.73
------------------------------
epoch #: 5
Train loss is: 0.79
Train accuracy is: 0.74
val loss is: 0.78
val accuracy is: 0.74
------------------------------
epoch #: 6
Train loss is: 0.72
Train accuracy is: 0.76
val loss is: 0.76
val accuracy is: 0.74
------------------------------
epoch #: 7
Train loss is: 0.66
Train accuracy is: 0.78
val loss is: 0.73
val accuracy is: 0.75
------------------------------
epoch #: 8
Train loss is: 0.56
Train accuracy is: 0.81
val loss is: 0.73
val accuracy is: 0.76
-----------------------

In [17]:
model.load_state_dict(torch.load('best_cnn_clf.pt'))
test_loss,test_acc = evaluate(model, test_iterator, criterion)
print('Test loss : {:.2f}'.format(test_loss))
print('Test acc : {:.2f}'.format(test_acc))

Test loss : 0.59
Test acc : 0.81


### Convolutional Neural Network 결과 요약
- Naive Bayesian에서 모델을 제외하고 Naive Baysian과 동일한 가정에서 CNN을 사용했을 경우 test set에서의 정확도는 약 79%
- 따라서 Neural method 중 하나인 CNN은 baseline인 Naive Baysian보다 4.2%p 높은 정확도를 갖는다고 결론

## User Input

####  뉴스 labels
    -  IT/과학': 0, '경제': 1, '문화': 2, '미용/건강': 3, '사회': 4, '생활': 5, '스포츠': 6, '연예': 7, '정치': 8

In [18]:
## 아래 문장의 정답은 8-4-1-3-5-6-0-2-7
## 연예/문화, 정치/경제/사회/생활 등 명확히 구별되기 어려운 범주들이 있음...


input_data= pd.DataFrame(["여러 차례 선거를 치르며 조직적인 지지모임과 온라인 팬덤을 보유한 이 지사에 비해 부족하다는 것이다. 윤석열 캠프에선 오차범위를 다투는 여론조사 지지율과는 별개로, ‘조직’에 있어선 아직도 채워야 할 부분이 많다는 이야기가 나온다. 윤석열 캠프 관계자는 “사람만 많이 모은다고 좋을 게 없다는 지적을 듣기도 하지만, 인구 1300만명의 지자체장인 이재명 지사에 비하면 많은 게 아닌 상황”이라고 말했다.", 
"여자친구가 이별을 통보하고 새 남자친구를 사귀자 지속적으로 찾아가 협박과 폭행을 가한 30대 남성이 실형을 선고 받았다. 창원지법 형사4단독 안좌진 부장판사는 상해, 주거침입, 폭행 등 혐의로 재판에 넘겨진 A(39)씨에게 징역 1년3개월을 선고했다고 10일 밝혔다.A씨는 지난해 4월부터 올해 2월까지 10개월 가량 사귄 B씨(30)가 이별을 통보하자 지난 3월 6일 B씨 집을 찾아가 욕설을 퍼붓고 B씨를 폭행한 혐의를 받고 있다.",
"동탄신도시의 성공은 명실상부한 한국 1위의 기업 삼성전자를 빼놓고는 설명할 수 없다. 삼성전자는 수출의 20%를 담당하는 한국 경제의 심장이다. 삼성전자의 연구소와 공장은 세계 최고 수준의 연구인력과 협력업체를 끌어당기는 블랙홀이다.동탄신도시 인근에 삼성전자 기흥캠퍼스가 있고 화성캠퍼스가 신도시에 자리 잡고 있다. 삼성 화성캠퍼스에서는 메모리와 파운드리 반도체의 설계 및 생산이 이뤄지고 있다.",
"샤워나 목욕 중에는 물, 샤워타올, 수건 등 균이 닿을 여지가 많다. 샤워를 하는 화장실에는 보통 변기도 함께 있어 배변 활동으로 나온 균이 공기 중을 돌아다니고 있다. 습기가 높아 곰팡이가 생기기도 좋은 환경이다. 화장실에 걸린 샤워타올과 수건이 제대로 건조되지 않은 채 화장실에 내내 있었다면 균이 있을 가능성이 크다. 이 균이 예방 접종 하면서 생긴 손상 부위에 닿으면 드물지만 침입해 감염증을 유발할 수 있다.",
"식전주의 시간이다. 밥을 먹기 전에 마시는 술. 안주와 함께 먹지 않는 술. 술만으로 온전한 술. 이게 식전주다. 3시와 5시 사이는 식전주의 시간이기도 한 것이다. 이 시간에 마시는 식전주를 나는 꽤나 좋아한다. 술은 다 각각의 매력이 있고, 슬플 때도 기쁠 때도 지루할 때도 피곤할 때도 좋지만, 식전주의 시간에 마시는 식전주도 좋다. 주로 맥주이지만 가끔은 아페리티프(Aperitif·식전주)를 마신다.",
"시리아전을 마친 뒤 9일 이란으로 출국한 한국 대표팀은 한국 시간 기준 10일 오전 1시경 테헤란 공항에 도착해 숙소인 파르시안 아자디 호텔로 이동했다. 이후 코로나19 PCR 검사를 진행했고, 결과가 나올 때까지 각자 방에서 격리한 채 대기할 예정이다. 한국은 역대 이란 원정에서 한차례도 승리하지 못한 채 2무 5패를 기록 중이다.  선수들이 좋은 컨디션을 유지할 수 있도록 전세기를 마련해 이란으로 향했다.",
"애플의 아이폰13 시리즈가 지난 8일 국내 판매를 시작했다. 애플이 지난달 14일(현지시각) 신제품을 공개한 후 3주 만이다. 애플은 아이폰13의 두뇌에 해당하는 프로세서와 카메라 성능을 크게 개선했다고 밝혔다. 팀 쿡 애플 최고경영자(CEO)는 “역사상 최고의 아이폰이다”라고 했다. 하지만 전작인 아이폰12와 비교해 큰 차이를 느낄 수 없다는 부정적인 평가도 많다.",
"극단 마실은 문화체육관광부와 지역문화진흥원 지원으로 '심청전-할머니의 비밀레시피' 온라인 만남 행사를 진행했다고 10일 밝혔다. 행사는 할머니만의 레시피로 함께 음식을 만들며 할머니의 이야기를 공유하고, 할머니를 주인공으로 한 짤막한 연극을 펼치는 순서로 진행됐다. 극단은 지역 내 관음사 연기 설화가 심청전과 연관 있는 점을 토대로 심청의 일생과 닮은 곡성 할머니들의 이야기를 2018년도부터 수집해 연극을 만들었다.",
"‘놀면 뭐하니?+’에서는 유재석, 정준하, 하하, 신봉선, 미주의 깜짝 ‘꼬치꼬치 기자간담회’와 MBC 보도국 열혈 신입기자로 변신한 ‘뉴스데스크’ 특집이 시작됐다. ‘꼬치꼬치 기자간담회’에서는 정준하가 ‘스포츠 꼬치꼬치’ 기자로 변신, 시청자의 궁금증을 풀어주는 마성의 돌직구 질문을 던졌고, ‘놀면 뭐하니?+’ 멤버들은 솔직한 마음이 담긴 답변으로 큰 웃음과 훈훈함을 동시에 선사했다."
])
input_label = pd.DataFrame(["정치", "사회", "경제", "미용/건강", "생활", "스포츠", "IT/과학", "문화", "연예"])

In [19]:
def predict_news(model, sentence, min_length=5):
    model.eval()
    text_tokenized=tokenizer.morphs(sentence)
    if len(text_tokenized) < min_length:
        text_tokenized += ['<pad>'] * (min_length - len(text_tokenized))
    padded_text_tokenized = [TEXT.vocab.stoi[t] for t in text_tokenized]
    tensor = torch.LongTensor(padded_text_tokenized).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = model(tensor).argmax(1)
    label_index = prediction.item()
    label_string = list(LABEL.vocab.stoi)[label_index]
    return label_string
    

In [20]:
#Naive Bayes for comaprison
NB_pred=NB_split.predict(vectorizer.transform(input_data[0]))
print('Naive Bayes Results')
print('Prediction results: {0}'.format(NB_pred))
print('Accuracy: {0}'.format(accuracy_score(NB_pred, input_label)))

Naive Bayes Results
Prediction results: ['정치' '사회' 'IT/과학' '미용/건강' '미용/건강' '스포츠' 'IT/과학' '문화' '연예']
Accuracy: 0.7777777777777778


In [22]:
#Convolutional Neural Network Model
CNN_pred=[]
for sentence in input_data[0]:
    pred=predict_news(model, sentence)
    CNN_pred.append(pred)
print('CNN Results')
print('Prediction results: {0}'.format(CNN_pred))
print('Accuracy: {0}'.format(accuracy_score(CNN_pred, input_label)))  

CNN Results
Prediction results: ['사회', '사회', '경제', '미용/건강', '생활', '스포츠', 'IT/과학', '문화', '연예']
Accuracy: 0.8888888888888888


##  Conclusion

1. User Input 테스트에서 정의한 Naive Bayes는 7개를, CNN은 실험마다 다르게 5~8개의 label을 옳게 예측하였다.
2. 그런데 Naive Bayes는 원래 학습 결과 정확도가 75%정도 나왔고, CNN은 80%가량 나왔었다.
3. 이로부터 알 수 있는 것은 새로운 데이터에 대해 Naive Bayes는 Robustness가 확인된 반면, CNN은 그렇지 못하였다는 것이다.
3. 따라서 본 실험에서 내린 결론은 CNN 모델이 상황에 따라서 과적합이 될 수도 있기 때문에 정규화가 더 필요하거나 모델 자체의 개선이 필요하다는 것이다